In [2]:
import napari
import numpy as np

from openst.utils.points import split_points_by_categorical, point_inside_which_pointsets
from openst.alignment.pairwise_aligner import estimate_transform, apply_transform
from openst.utils.file import load_properties_from_adata

from skimage.transform import estimate_transform

# this one is optional, depending on whether we apply the fiducial correspondence
# from openst.alignment.fiducial_detection import correspondences_fiducials

In [4]:
ADATA_PATH = '/data/rajewsky/home/dleonpe/projects/openst_paper/data/2_downstream/fc_sts_63/auto_aligned_he/fc_sts_063_4_stitched_spots_aligned.h5ad'
ADATA_ALIGNMENT_UNS_PATH = 'spatial_pairwise_aligned'
ADATA_ALIGNMENT_UNS_PATH_REFINE = 'spatial_pairwise_aligned_refined'
OUTPUT_PATH = '/data/rajewsky/home/dleonpe/projects/openst_paper/data/2_downstream/fc_sts_63/auto_aligned_he/fc_sts_063_4_stitched_spots_aligned_refined.h5ad'

In [10]:
sts = load_properties_from_adata(ADATA_PATH, properties=["obsm/spatial_pairwise_aligned_fine", "obs/puck_id", "uns/spatial_pairwise_aligned/staining_image_transformed"])
staining_image = sts["uns/spatial_pairwise_aligned/staining_image_transformed"]

In [6]:
viewer = napari.Viewer()

In [11]:
sts_coords = sts["obsm/spatial_pairwise_aligned_fine"]
puck_id = sts["obs/puck_id"].codes
tiles = split_points_by_categorical(sts_coords, puck_id)

In [ ]:
viewer.add_image(staining_image)

In [16]:
for key, tile in tiles.items():
    viewer.add_points(
        tile[::10][:, ::-1],
        name=key,
        size=20,
        opacity=0.5
    )

In [26]:
points_fiducial_staining = np.array(viewer.layers[f"points_fiducial_staining"].data)
points_fiducial_sts = np.array(viewer.layers[f"points_fiducial_sts"].data)

In [27]:
adata_uns_refinement = {}
adata_uns_refinement['points_fiducial_staining'] = points_fiducial_staining
adata_uns_refinement['points_fiducial_sts'] = points_fiducial_sts

In [31]:
points_fiducial_sts, points_in_pointset

(array([[ 7691.74361017, 12378.6614719 ],
        [ 8641.29913774, 12512.18466446],
        [10325.14666383, 13906.60964168],
        [11099.28565735, 13940.58654097],
        [13875.12080311, 12365.08121116],
        [14807.28018893, 12468.99489708],
        [13570.90172214, 14807.27101611],
        [14501.1488476 , 14923.90750584]]),
 array([5, 5, 10, 10, None, None, 11, 11], dtype=object))

In [29]:
# this must be moved to the code (not the plugin)
points_in_pointset = point_inside_which_pointsets(points_fiducial_sts, tiles)
points_sts_fiducial_by_pointset = split_points_by_categorical(points_fiducial_sts, points_in_pointset)
points_staining_fiducial_by_pointset = split_points_by_categorical(points_fiducial_staining, points_in_pointset)

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [21]:
sts_coords_refined = sts_coords.copy()

In [ ]:
points_sts_matched = {}
points_staining_matched = {}

for puck in np.unique(puck_id):
    # The code below finds the mutually nearest points 
    # points_sts_matched[puck], points_staining_matched[puck] = correspondences_fiducials(points_sts_fiducial_by_pointset[puck], points_staining_fiducial_by_pointset[puck])
    # The code below will use the points based on which order they were selected
    points_sts_matched[puck], points_staining_matched[puck] = points_sts_fiducial_by_pointset[puck], points_staining_fiducial_by_pointset[puck]

    puck_tform = estimate_transform("similarity", points_sts_matched[puck], points_staining_matched[puck])
    
    sts_coords_refined[puck_id == puck] = apply_transform(
        tiles[puck], puck_tform, check_bounds=True
    )

In [ ]:
adata.write_h5ad(OUTPUT_PATH)